In [1]:
import pandas as pd
import numpy as np

In [2]:
train_data =  pd.read_csv("Dataset/Train.csv")
test_data = pd.read_csv("Dataset/Test.csv")
sample_sub = pd.read_csv("Dataset/Sample_Submission.csv")

In [4]:
print("train shape:", train_data.shape)
print("test shape:", test_data.shape)

train shape: (18288, 3)
test shape: (5112, 2)


In [5]:
train_data.head()

,ID,Datetime,Count
0,0,25-08-2012 00:00,8
1,1,25-08-2012 01:00,2
2,2,25-08-2012 02:00,6
3,3,25-08-2012 03:00,2
4,4,25-08-2012 04:00,2


In [7]:
X_train = train_data.Datetime.values
y_train = train_data.Count.values

In [8]:
X_train.shape

(18288,)

In [19]:
train= train_data.iloc[:, 2:3].values

In [20]:
X_train = []
y_train = []
for i in range(24, 18288):
    X_train.append(train[i-24:i, 0])
    y_train.append(train[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)

In [26]:
X_train.shape

(18264, 24, 1)

In [10]:
# Reshaping
X_train = np.reshape(X_train, (X_train.shape[0],1, 1))

In [11]:

# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

# Initialising the RNN
regressor = Sequential()

# Adding the first LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
regressor.add(Dropout(0.2))

# Adding a second LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a third LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a fourth LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))

# Adding the output layer
regressor.add(Dense(units = 1))

# Compiling the RNN
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

# Fitting the RNN to the Training set
regressor.fit(X_train, y_train, epochs = 10, batch_size = 32)

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Epoch 1/10


ValueError: could not convert string to float: '07-11-2013 13:00'

In [28]:
dataset_total = pd.concat((train_data['Count'], test_data['Count']), axis = 0)
inputs = dataset_total[len(dataset_total) - len(test_data) - 24:].values
inputs = inputs.reshape(-1,1)
X_test = []
for i in range(24, 5112):
    X_test.append(inputs[i-24:i, 0])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
predicted_stock_price = regressor.predict(X_test)

KeyError: 'Count'

In [ ]:
raw_data = {'ID' : sample_sub["ID"],
            'Count' : predicted_stock_price
            }
result_ = pd.DataFrame(raw_data)
result_.head()

In [ ]:
result_.to_csv("submission.csv",index=False)

In [30]:
test_data['Count']

KeyError: 'Count'